In [1]:
# pip install cupy

In [2]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import cupy as cp

In [21]:
test_df = pd.read_csv("/kaggle/input/30-days-of-ml/test.csv")
df = pd.read_csv("/kaggle/input/30-days-of-ml-5-folds/train_folds.csv")
# df.head()

features = [col for col in df.columns if col not in ('id', 'kfold', 'target')]
object_cols = [col for col in features if 'cat' in col]
xtest_df = test_df[features].copy()

for col in object_cols:
    temp_df = []
    temp_test_feat = None
    for fold in range(5):
        xtrain = df.loc[df.kfold != fold].reset_index(drop=True)
        xvalid = df.loc[df.kfold == fold].reset_index(drop=True)    
        feat = xtrain.groupby(col)['target'].agg('mean')
        feat = feat.to_dict()
        xvalid.loc[:, f"tar_enc_{col}"] = xvalid[col].map(feat)
        temp_df.append(xvalid)
        if temp_test_feat is None:
            temp_test_feat = xtest_df[col].map(feat)
        else:
            temp_test_feat = temp_test_feat + xtest_df[col].map(feat)
        
    temp_test_feat = temp_test_feat / 5
    xtest_df.loc[:, f"tar_enc_{col}"] = temp_test_feat
    df = pd.concat(temp_df, axis=0)
#     print(df)
#     break

In [23]:
df.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,tar_enc_cat0,tar_enc_cat1,tar_enc_cat2,tar_enc_cat3,tar_enc_cat4,tar_enc_cat5,tar_enc_cat6,tar_enc_cat7,tar_enc_cat8,tar_enc_cat9
0,1,B,B,B,C,B,B,A,E,C,...,8.245979,8.203850,8.224780,8.236717,8.240572,8.229516,8.240567,8.240285,8.280709,8.249782
1,8,B,A,A,A,B,D,A,E,C,...,8.245979,8.276689,8.244491,8.274495,8.240572,8.250754,8.240567,8.240285,8.280709,8.259165
2,13,A,B,A,C,B,B,A,E,A,...,8.238970,8.203850,8.244491,8.236717,8.240572,8.229516,8.240567,8.240285,8.230681,8.249782
3,14,B,B,A,C,B,D,A,E,C,...,8.245979,8.203850,8.244491,8.236717,8.240572,8.250754,8.240567,8.240285,8.280709,8.234356
4,25,B,B,A,C,B,D,A,E,C,...,8.245979,8.203850,8.244491,8.236717,8.240572,8.250754,8.240567,8.240285,8.280709,8.259165


In [25]:
features = [col for col in df.columns if col not in ('id', 'kfold', 'target')]
object_cols = [col for col in features if col.startswith('cat')]
xtest_df = xtest_df[features]

In [27]:

final_predictions = []
scores = []
for fold in range(5):
    xtrain = df.loc[df.kfold != fold].reset_index(drop=True)
    xvalid = df.loc[df.kfold == fold].reset_index(drop=True)
    xtest = xtest_df.copy()
    ytrain = xtrain.target
    yvalid = xvalid.target

    xtrain = xtrain[features]
    xvalid = xvalid[features]

    print("Encoding")
    ordinal_encoder = preprocessing.OrdinalEncoder()
    xtrain[object_cols] = ordinal_encoder.fit_transform(xtrain[object_cols])
    xvalid[object_cols] = ordinal_encoder.transform(xvalid[object_cols])
    xtest[object_cols] = ordinal_encoder.transform(xtest[object_cols])

    print("Training")
    model = xgb.XGBRegressor(random_state=fold, tree_method='hist', device='cuda')
    # Load training and validation data onto GPU (if necessary)
    X_train_gpu = cp.array(xtrain)
    y_train_gpu = cp.array(ytrain)
    
    model.fit(X_train_gpu, y_train_gpu)
      # Handle potential device mismatch for prediction
    if model.device != 'cpu':
        model.set_params(device='cpu')    
    
    preds_valid = model.predict(xvalid)
    preds_test = model.predict(xtest)
    final_predictions.append(preds_test)
    rmse = mean_squared_error(preds_valid, yvalid, squared=False)
    scores.append(rmse)
    print(f"Fold_{fold+1} |", "RMSE:", rmse)

print("Mean RMSE:", np.mean(scores), "| Scores STD:", np.std(scores))

Encoding
Training
Fold_1 | RMSE: 0.7257139401614819
Encoding
Training
Fold_2 | RMSE: 0.724866165621642
Encoding
Training
Fold_3 | RMSE: 0.72629803201182
Encoding
Training
Fold_4 | RMSE: 0.7279330306996882
Encoding
Training
Fold_5 | RMSE: 0.7605979069431025
Mean RMSE: 0.7330818150875469 | Scores STD: 0.013794545890077784
